In [117]:
import PyPDF2 as pdf
from tabula import read_pdf
from tabulate import tabulate
import pandas as pd
import os
import re

In [139]:
def gener_gene_data(pdf_path):
    pdf = read_pdf(pdf_path, pages=1)
    pd_df = pdf[1]
    pd_df = pd_df.rename(columns=pd_df.iloc[0]).drop(pd_df.index[0])

    id_list = list(pd_df["TRANSCRIPT ID"])
    id_list = [x.split('.')[0] for x in id_list]
    patient_name = '-'.join(pdf_path.split('_')[:2])
    
    print("Patient Name: " + patient_name)
    print("Transcript Ids being retrieved: " + str(id_list))
    
    xl_files_og = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('.xlsx')]
    xl_files = [f.replace('%', '_') for f in xl_files_og]
    xl_files = [f.split('_')[0] for f in xl_files]
    ind = 0
    for i in range(0, len(xl_files)):
        if patient_name == xl_files[i]:
            ind = i
    

    print("Reading from " + xl_files_og[ind])
    gene_data = pd.read_excel(xl_files_og[ind])
    # gene_data["Variant Filter"] = gene_data.apply(lambda s: s.Variant.split(' ')[0], axis=1)
    gd = gene_data.loc[gene_data["TranscriptID"].isin(id_list)]
    print("table made")
    return gd  

# gener_gene_data("CCGL_3205_FinalReport_pedi_LFS_associated_diffuse_astrocytoma_IDH1_R132H_germline_TP53.pdf")

files = [f for f in os.listdir('.') if os.path.isfile(f) and f.endswith('.pdf')]
print("using the following pdf files:")
print(files)
data_reports = {f: gener_gene_data(f) for f in files}
# data_reports



using the following pdf files:
['CCGL_3152_FinalReport_Sandler_adult_GBM_IDH1_R132H_ATRX_CDKN2A__no_TP53_.pdf', 'CCGL_3149_FinalReport_adult_recurrent_astrocytoma_IDH1_R132H_TP53_ATRX_CDKN2A_PDGFRA_amp.pdf', 'CCGL_2831_FinalReport_adult_astrocytoma_IDH1_R132H_TP53_ATRX.pdf', 'CCGL_2835_FinalReport_adult_astrocytoma_IDH1_R132H_ATRX_TP53_PIK3R1.pdf', 'CCGL_3205_FinalReport_pedi_LFS_associated_diffuse_astrocytoma_IDH1_R132H_germline_TP53.pdf', 'CCGL_3619_FinalReport_adult_Ollier_associated_astrocytoma_IDH1_R132H_TP53_ATRX_NF1_SPRED1_NOTCH1_constitutional_mosaicism_IDH1.pdf']
Patient Name: CCGL-3152
Transcript Ids being retrieved: ['NM_000489', 'all', 'NM_005896']
Reading from CCGL-3152_CGP-9473_T_signout.xlsx
table made
Patient Name: CCGL-3149
Transcript Ids being retrieved: ['NM_000489', 'all', 'NM_005896', 'all', 'NM_000546']
Reading from CCGL-3149_CGP-9470_T_signout.xlsx
table made
Patient Name: CCGL-2831
Transcript Ids being retrieved: ['NM_000489', 'NM_005896', 'NM_000546']
Reading f

In [135]:
a = data_reports.get("CCGL_2835_FinalReport_adult_astrocytoma_IDH1_R132H_ATRX_TP53_PIK3R1.pdf")
a

,CHROM,POS,REF,ALT,Func,T:FUP,Variant,TranscriptID,Classification,T:COV,...,gnomADexome,esp6500si,1000g2015aug,snp150,ucsf500normT,ucsf500normN,cosmic87,cbioportal2018nov,clinvar,CADD
67,chrX,76937963,G,C,exonic,110,ATRX p.Q929E,NM_000489,NaN,415,...,37.3300%,50.8300%,55.2318%,rs3088074,34.5560%,34.5560%,"ID=COSM3759543,COSM3759544;OCCURRENCE=2(breast...",4:ATRX:Q929E(4),CLINSIG=Benign;CLNDN=not_specified;CLNREVSTAT=...,0.391
69,chr17,7579472,G,C,exonic,110,TP53 p.P72R,NM_000546,NaN,409,...,66.8600%,63.0000%,54.2931%,rs1042522,69.3880%,69.3610%,"ID=COSM250061,COSM3766193,COSM3766192,COSM3766...",33:TP53:P72R(33),CLINSIG=drug_response;CLNDN=CODON_72_POLYMORPH...,8.316
78,chr17,7578407,G,C,exonic,110,TP53 p.R175G,NM_000546,NaN,304,...,NaN,NaN,NaN,rs138729528,NaN,NaN,"ID=COSM4070040,COSM707882,COSM707883,COSM16408...",21:TP53:R175G(21),CLINSIG=Pathogenic/Likely_pathogenic;CLNDN=Med...,18.190
105,chr5,67591104,TTAA,T,exonic,101,PIK3R1 p.566_567del,NM_181523,NaN,752,...,NaN,NaN,NaN,NaN,NaN,NaN,"ID=COSM5835073,COSM5835072,COSM5835074;OCCURRE...",3:PIK3R1:K567del(3),NaN,NaN
202,chrX,76939929,C,CCT,exonic,111,ATRX p.L273fs,NM_000489,NaN,375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
pdfFile = open("CCGL_2831_FinalReport_adult_astrocytoma_IDH1_R132H_TP53_ATRX.pdf", "rb")
pdf = read_pdf("CCGL_2831_FinalReport_adult_astrocytoma_IDH1_R132H_TP53_ATRX.pdf", pages="all")
pd_df = pdf[1]

In [79]:
pd_df = pd_df.rename(columns=pd_df.iloc[0]).drop(pd_df.index[0])

In [80]:
pd_df

,VARIANT,TRANSCRIPT ID,CLASSIFICATION,READS,MUTANT ALLELE\rFREQUENCY
1,ATRX p.M1679fs,NM_000489.3,Pathogenic,379,51%
2,IDH1 p.R132H,NM_005896.2,Pathogenic,780,25%
3,TP53 p.R273C,NM_000546.5,Pathogenic,334,54%


In [57]:
variant_list = [x.split(' ')[0] for x in list(pd_df["VARIANT"])]

['ATRX', 'IDH1', 'TP53']

In [81]:
id_list = list(pd_df["TRANSCRIPT ID"])
id_list = [x.split('.')[0] for x in id_list]
id_list

['NM_000489', 'NM_005896', 'NM_000546']

In [85]:
gene_data = pd.read_excel("CCGL-2831_CGP-8784_T_signout.xlsx")
# gene_data["Variant Filter"] = gene_data.apply(lambda s: s.Variant.split(' ')[0], axis=1)
gd = gene_data.loc[gene_data["TranscriptID"].isin(id_list)]
gd

,CHROM,POS,REF,ALT,Func,T:FUP,Variant,TranscriptID,Classification,T:COV,...,gnomADexome,esp6500si,1000g2015aug,snp150,ucsf500normT,ucsf500normN,cosmic87,cbioportal2018nov,clinvar,CADD
63,chr17,7579472,G,C,exonic,110,TP53 p.P72R,NM_000546,NaN,336,...,66.8600%,63.0000%,54.2931%,rs1042522,69.3880%,69.3610%,"ID=COSM250061,COSM3766193,COSM3766192,COSM3766...",33:TP53:P72R(33),CLINSIG=drug_response;CLNDN=CODON_72_POLYMORPH...,8.316
64,chr17,7577121,G,A,exonic,110,TP53 p.R273C,NM_000546,NaN,334,...,0.0012%,NaN,NaN,rs121913343,NaN,NaN,"ID=COSM3355991,COSM99933,COSM10659,COSM1645518...",429:TP53:R273C(429),CLINSIG=Conflicting_interpretations_of_pathoge...,17.480
182,chrX,76888758,GAGCAAGATTTCTATACATCTCATAGCCTATGATCATA,G,exonic,101,ATRX p.V1678fs,NM_000489,NaN,379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pdfReader = pdf.PdfReader(pdfFile)

In [16]:
print(pdfReader)

In [19]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.
